In [1]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev # 오류 해결 명령어

!pip3 install JPype1-py3 # 파이썬 언어와 자바 언어의 중간 다리 JPype1-py3
!pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [634 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,326 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,305 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,046 kB]
Get:13 http

In [2]:
# 유저 input과 유사한 기업 내 키워드 제시 w/ 빈도

from gensim.models.word2vec import Word2Vec
from konlpy.tag import Hannanum, Okt
from tqdm import tqdm

In [3]:
# 전체 데이터
import pandas as pd

data = pd.read_csv('[32]total_40_spell.csv')
data['spell'] = data['spell'].astype(str)

In [4]:
# 추가 전처리
data['spell'] = data['spell'].apply(lambda x:x.replace('오라 밸', '워라밸'))
data['spell'] = data['spell'].apply(lambda x:x.replace('워라밸만', '워라밸'))
data['spell'] = data['spell'].apply(lambda x:x.replace('워라밸이', '워라밸'))
data['spell'] = data['spell'].apply(lambda x:x.replace('워라밸은', '워라밸'))

data['spell'] = data['spell'].apply(lambda x:x.replace('커리어', '커리어'))
data['spell'] = data['spell'].apply(lambda x:x.replace('커리', '커리어'))
data['spell'] = data['spell'].apply(lambda x:x.replace('커리 어', '커리어'))
data['spell'] = data['spell'].apply(lambda x:x.replace('직업', '커리어'))

data['spell'] = data['spell'].apply(lambda x: x.replace('비젼', '비전'))
data['spell'] = data['spell'].apply(lambda x: x.replace('그냥저냥', '그저 그런'))
data['spell'] = data['spell'].apply(lambda x: x.replace('걍', '그냥'))
data['spell'] = data['spell'].apply(lambda x: x.replace('캐이비', 'KB'))
data['spell'] = data['spell'].apply(lambda x: x.replace('It', 'IT'))

data['spell'] = data['spell'].apply(lambda x: x.replace('부바부', '부서 간 차이 있음'))
data['spell'] = data['spell'].apply(lambda x: x.replace('부마부', '부서 간 차이 있음'))

data['spell'] = data['spell'].apply(lambda x: x.replace('개바개', '개인 간 차이 있음'))
data['spell'] = data['spell'].apply(lambda x: x.replace('팀바팀', '팀별 차이 있음'))
data['spell'] = data['spell'].apply(lambda x: x.replace('케바케', '상황마다 다름'))
data['spell'] = data['spell'].apply(lambda x: x.replace('캐바캐', '상황마다 다름'))
data['spell'] = data['spell'].apply(lambda x: x.replace('사바사', '사람마다 다름'))

data['spell'] = data['spell'].apply(lambda x: x.replace('빵빵', '넉넉함'))
data['spell'] = data['spell'].apply(lambda x: x.replace('so so', '보통'))
data['spell'] = data['spell'].apply(lambda x: x.replace('soso', '보통'))
data['spell'] = data['spell'].apply(lambda x: x.replace('쏘쏘', '보통'))
data['spell'] = data['spell'].apply(lambda x: x.replace('쏘소', '보통'))
data['spell'] = data['spell'].apply(lambda x: x.replace('소쏘', '보통'))

data['spell'] = data['spell'].apply(lambda x: x.replace('고인 물', '고인물'))
data['spell'] = data['spell'].apply(lambda x: x.replace('워러벨', '워라밸'))
data['spell'] = data['spell'].apply(lambda x: x.replace('워라벨', '워라밸'))
data['spell'] = data['spell'].apply(lambda x: x.replace('워라맬', '워라밸'))
data['spell'] = data['spell'].apply(lambda x: x.replace('워라멜', '워라밸'))
data['spell'] = data['spell'].apply(lambda x: x.replace('워리벨', '워라밸'))
data['spell'] = data['spell'].apply(lambda x: x.replace('워리밸', '워라밸'))
data['spell'] = data['spell'].apply(lambda x: x.replace('와 라벨', '워라밸'))
data['spell'] = data['spell'].apply(lambda x: x.replace('와 라밸', '워라밸'))

In [36]:
# 하나의 회사 리뷰 데이터를 Word2Vec 모델에 학습시킵니다

# 유저가 회사에 궁금한 키워드를 찾을 경우
# 해당 회사 리뷰에서 입력값과 비슷한 키워드를 찾습니다

# 상위 3개의 키워드를 알려주고
# 각 키워드가 포함된 리뷰의 평균 평점을 계산합니다.

def user_input_similar_keywords(company, user_input): # company: 회사 이름, user_input: 찾고 싶은 키워드
  from gensim.models import Word2Vec

  with open('./[30]stopword_ko.txt') as f:
    stop_words = f.read().splitlines()
    f.close()

  stop_words.append('곳')
  stop_words.append('편')
  stop_words.append('수')
  stop_words.append('리')
  stop_words.append('중')
  stop_words.append('회사')
  stop_words.append('기업')

  data_to_use = data[data['name']==company]
  mode = Hannanum()

  tokenized_words = []
  for sent in tqdm(data_to_use['spell']):
    sent = mode.pos(sent)
    stop_sent = [word for word in sent if word[0] not in stop_words]
    n_sent = [word[0] for word in stop_sent if word[1].startswith('N') or word[1].startswith('V')]
    tokenized_words.append(n_sent)

  model = Word2Vec(sentences = tokenized_words, vector_size = 3000, window = 10, min_count = 2, workers = 4, sg = 0)

  word_lst = [similar_word[0] for similar_word in model.wv.most_similar(user_input)[:3]]
  word_lst.insert(0, user_input)

  first_word = set(data_to_use[data_to_use['spell'].str.contains(word_lst[0])].index)
  second_word = set(data_to_use[data_to_use['spell'].str.contains(word_lst[1])].index)
  third_word = set(data_to_use[data_to_use['spell'].str.contains(word_lst[2])].index)

  all_word = list(first_word.union(second_word, third_word))

  word1_data = data_to_use.loc[sorted(first_word)]['rating'].mean()
  word2_data = data_to_use.loc[sorted(second_word)]['rating'].mean()
  word3_data = data_to_use.loc[sorted(third_word)]['rating'].mean()
  all_word_data = data_to_use.loc[sorted(all_word)]['rating'].mean()

  similar_word = pd.DataFrame(columns=['단어', '평점'])
  similar_word['단어'] = word_lst
  similar_word['평점'] = [all_word_data, word1_data, word2_data, word3_data]

  return similar_word

In [37]:
# user_input_similar_keywords(회사이름, 원하는 키워드)
# ['cjENM', 'cj제일제당', 'sk하이닉스', '네이버', '대한항공', '삼성물산', '삼성전자','아모레퍼시픽', '카카오', '현대자동차']

user_input_similar_keywords('네이버', '성장')

100%|██████████| 3178/3178 [00:05<00:00, 562.92it/s]


,단어,평점
0,성장,4.138425
1,업무,4.223404
2,문화,3.894737
3,직원,4.000000


In [38]:
user_input_similar_keywords('네이버', '커리어')

100%|██████████| 3178/3178 [00:04<00:00, 736.58it/s] 


,단어,평점
0,커리어,4.024306
1,개발,4.007692
2,워라밸,4.217054
3,보상,3.974359


In [39]:
user_input_similar_keywords('네이버', '문화')

100%|██████████| 3178/3178 [00:04<00:00, 726.13it/s]


,단어,평점
0,문화,3.826172
1,워라밸,4.000000
2,보상,3.974359
3,직원,3.075269


In [40]:
user_input_similar_keywords('삼성전자', '커리어')

100%|██████████| 9990/9990 [00:11<00:00, 897.29it/s] 


,단어,평점
0,커리어,3.689136
1,워라밸,3.627986
2,가능,3.684685
3,업무,3.796117


In [41]:
user_input_similar_keywords('sk하이닉스', '커리어')

100%|██████████| 4830/4830 [00:07<00:00, 641.38it/s] 


,단어,평점
0,커리어,1.969283
1,경영진,3.480000
2,구성원,1.834711
3,반도체,1.484848
